# Data Collector

Three data sets are required for each environment 
* __Expert dataset (D1)__ (generated by an agent that solves the environment)
* __Suboptimal dataset (D2)__ (generated by an agent showing suboptimal results)
* __Random dataset (D3)__

In [1]:
import os
import numpy as np
from stable_baselines import PPO2
from stable_baselines.gail import generate_expert_traj
from BEAR.utils import ReplayBuffer

## Collecting a expert off-policy dataset
Erklärung hier...

In [2]:
# Set Parameters
dataste_n_episodes = 1e2 # number of episodes in the dataset 
train_n_steps = 1e2 # number of tranings steps befor collecting data 
env_name = 'Pendulum-v0' # name of the enviornment

# Set path
steps_str = ('%.E' % train_n_steps).replace("+","") 
dataset_name = env_name+'_expert_'+steps_str # name of the dataset
dir_name = "./data/expert_traj/" # path to the directory
path = os.path.join(dir_name, dataset_name)

# make sure the directory exists
if not os.path.exists(dir_name):
    os.makedirs(dir_name)

print('path:' + path + '\n')
    
# Set Parameters
dataste_n_episodes = 1e2 # number of episodes in the dataset 
train_n_steps = 1e2 # number of tranings steps befor collecting data 

# Collect dataset
model = PPO2('MlpPolicy', env_name, verbose=0)
generate_expert_traj(model, path, n_timesteps=int(train_n_steps), n_episodes=int(dataste_n_episodes))

W0507 11:04:50.742345 140550023841600 deprecation_wrapper.py:119] From /home/uvegz/envs/venv_mujoco/lib/python3.6/site-packages/stable_baselines/common/tf_util.py:191: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0507 11:04:50.742906 140550023841600 deprecation_wrapper.py:119] From /home/uvegz/envs/venv_mujoco/lib/python3.6/site-packages/stable_baselines/common/tf_util.py:200: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0507 11:04:50.748330 140550023841600 deprecation_wrapper.py:119] From /home/uvegz/envs/venv_mujoco/lib/python3.6/site-packages/stable_baselines/common/policies.py:116: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0507 11:04:50.749042 140550023841600 deprecation_wrapper.py:119] From /home/uvegz/envs/venv_mujoco/lib/python3.6/site-packages/stable_baselines/common/input.py:25: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder in

path:./data/expert_traj/Pendulum-v0_expert_1E02



W0507 11:04:51.031216 140550023841600 deprecation_wrapper.py:119] From /home/uvegz/envs/venv_mujoco/lib/python3.6/site-packages/stable_baselines/ppo2/ppo2.py:190: The name tf.summary.scalar is deprecated. Please use tf.compat.v1.summary.scalar instead.

W0507 11:04:51.047035 140550023841600 deprecation.py:323] From /home/uvegz/envs/venv_mujoco/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0507 11:04:51.188100 140550023841600 deprecation_wrapper.py:119] From /home/uvegz/envs/venv_mujoco/lib/python3.6/site-packages/stable_baselines/ppo2/ppo2.py:206: The name tf.train.AdamOptimizer is deprecated. Please use tf.compat.v1.train.AdamOptimizer instead.

W0507 11:04:51.303235 140550023841600 deprecation_wrapper.py:119] From /home/uvegz/envs/venv_mu

actions (20000, 1)
obs (20000, 3)
rewards (20000, 1)
episode_returns (100,)
episode_starts (20000,)


{'actions': array([[ 2.6707017e-01],
        [ 2.4451163e-01],
        [ 5.8600211e-01],
        ...,
        [-1.1299404e+00],
        [-3.3715737e-01],
        [-1.0659453e-03]], dtype=float32),
 'obs': array([[-0.5745079 , -0.818499  , -0.5039217 ],
        [-0.6177589 , -0.78636754, -1.0777354 ],
        [-0.679757  , -0.73343736, -1.6308343 ],
        ...,
        [-0.85701793,  0.5152866 , -3.384071  ],
        [-0.7650374 ,  0.6439858 , -3.167097  ],
        [-0.67011625,  0.74225616, -2.7346814 ]], dtype=float32),
 'rewards': array([[-4.790077],
        [-5.118975],
        [-5.640485],
        ...,
        [-7.907774],
        [-6.966019],
        [-6.061619]], dtype=float32),
 'episode_returns': array([-1400.61669922, -1166.52453613, -1032.36010742, -1119.93310547,
        -1565.24047852,  -805.77789307, -1072.70117188, -1702.51831055,
        -1281.86682129, -1285.95861816, -1164.44848633, -1324.03234863,
         -874.5345459 , -1055.77038574,  -881.9609375 ,  -917.78863525

## Transform a NPZ in a ReplayBuffer

In [3]:
# set path to NPZ file
load_path = os.path.normpath('./data/expert_traj/Pendulum-v0_expert_1E02.npz')
print(load_path)

# Set path
dataset_name = (load_path.split('/')[-1]).split('.')[0]
dir_name = "./data/buffers/" # path to the directory
save_path = os.path.join(dir_name, dataset_name)

# make sure the directory exists
if not os.path.exists(dir_name):
    os.makedirs(dir_name)

# load data
data = np.load(load_path, allow_pickle=True)
obs = data['obs'] # shape = (878987, 4)
actions = data['actions'] # shape = (878987, 1)
rewards = data['rewards'] # shape = (878987, )
episode_starts = data['episode_starts']

# set ReplayBuffer
obs_dim = obs.shape[1]
actions_dim = actions.shape[1]
size = actions.shape[0]
replay_buffer = ReplayBuffer(obs_dim, actions_dim, size)

# Transform to ReplayBuffer
for i in range(len(obs)-2):
    if episode_starts[i+1]!=True:
        state = obs[i]
        next_state = obs[i+1]
        action = actions[i]
        reward = rewards[i]
        if episode_starts[i+2]!=True:
            done = True
        else:
            done = False
    data = (state, next_state, action, reward, done)
    replay_buffer.add(data)
    
# save ReplayBuffer    
replay_buffer.save(save_path)

data/expert_traj/Pendulum-v0_expert_1E02.npz


In [4]:
s, s2, a, r, d, mask = replay_buffer.sample(5)
s

array([[ 0.9904904 ,  0.13758214,  1.6978359 ],
       [ 0.6968495 ,  0.7172174 , -1.7568618 ],
       [ 0.33129063, -0.9435288 , -1.9987622 ],
       [-0.5448447 , -0.838537  ,  2.7617214 ],
       [ 0.15353766,  0.9881428 ,  4.9368424 ]], dtype=float32)

In [5]:
replay_buffer.sample(2)

(array([[-0.12015381, -0.9927553 , -3.477377  ],
        [-0.9777932 , -0.20957206, -6.9724307 ]], dtype=float32),
 array([[-0.31934118, -0.9476398 , -4.0917873 ],
        [-0.99076784,  0.1355695 , -6.9425106 ]], dtype=float32),
 array([[0.8677073],
        [1.2473264]], dtype=float32),
 array([[ -4.0702643],
        [-13.4506035]], dtype=float32),
 array([[1.],
        [1.]], dtype=float32),
 array([[0., 0., 0., 0.],
        [0., 0., 0., 0.]], dtype=float32))